In [548]:
import nltk
import math
import sys
import PyPDF2
import glob
import docx2txt
import string
import StringIO
from nltk.corpus import stopwords
from stemming.porter2 import stem
from glob import glob
from os.path import join, abspath
from os import listdir, getcwd
from operator import itemgetter 
stopWords = set(stopwords.words('english'))

In [549]:
import glob
worddokumen = glob.glob("dokumen/*.docx")
pdfdokumen = glob.glob("dokumen/*.pdf")
txtdokumen = glob.glob("dokumen/*.txt")

In [550]:
dokumen = {}

In [551]:
for i in worddokumen:
    text = docx2txt.process(i)
    dokumen[i[8:]] = text
for i in txtdokumen:
    file = open(i, 'r')
    dokumen[i[8:]] = file.read()
for i in pdfdokumen:
    pdfFileObj = open(i, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    page = pdfReader.numPages
    for k in range(0,page):
        pageObj = pdfReader.getPage(0)
        pageObj.extractText()
        dokumen[i[8:]] = pageObj.extractText()

In [552]:
dokumen.keys()

['tes.docx', 'memasak.txt', 'tes.pdf']

In [553]:
dokumen.values()

[u'I want you to be my girl.',
 'I want guys, and i will always love you. fuck you',
 u'Cause love never be so good. Oh yeah\n \n']

In [554]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in string.punctuation)

In [555]:
for key, value in (sorted(dokumen.items(), key = itemgetter(0))):
    dokumen[key] = strip_punctuation(dokumen[key])
    dokumen[key] = dokumen[key].lower()

In [556]:
dokumen.values()

[u'i want you to be my girl',
 'i want guys and i will always love you fuck you',
 u'cause love never be so good oh yeah\n \n']

In [557]:
index = {}
katak = {}

In [558]:
for key, value in (sorted(dokumen.items(), key = itemgetter(0))):
    kata = value.split()
    for i in kata:
        index[i.encode("utf-8")] = []
        katak[i.encode("utf-8")] = []

In [559]:
proxindex = {}

In [560]:
for key, value in (sorted(dokumen.items(), key = itemgetter(0))):
    kata = value.split()
    for i in kata:
        proxindex[i.encode("utf-8")] = []
    indeks=1
    for i in kata:
        proxindex[i.encode("utf-8")].append(indeks)
        indeks+=1
    for i in proxindex.keys():
        index[i.encode("utf-8")].append((key,proxindex[i.encode("utf-8")]))
        #index[i.encode("utf-8")] = {key:proxindex[i.encode("utf-8")]}
    proxindex.clear()

In [561]:
from collections import defaultdict
tfperdoc = defaultdict(dict)

In [562]:
for key, value in (sorted(dokumen.items(), key = itemgetter(0))):
    kata = value.split()
    temp = {}
    index = 1
    for i in kata:
        temp[i.encode("utf-8")] = []
    for i in kata:
        temp[i.encode("utf-8")].append(index)
        index+=1
    for i in kata:
        tfperdoc[key][i.encode("utf-8")] = temp[i.encode("utf-8")]

In [586]:
#Enter a query
query = 'i want you'

In [587]:
search = query.split()

In [588]:
scoreweight = {}
dfbigram = {}

In [589]:
tfperdoc['memasak.txt']

{'always': [7],
 'and': [4],
 'fuck': [10],
 'guys': [3],
 'i': [1, 5],
 'love': [8],
 'want': [2],
 'will': [6],
 'you': [9, 11]}

In [590]:
for i in range(1,len(search)):
        dfbigram[search[i-1]+search[i]] = 0

In [591]:
for key, value in (sorted(dokumen.items(), key = itemgetter(0))):
    lokasi = {}
    kata = value.split()
    index =1 
    for i in kata:
        lokasi[index] = i
        index+=1
    for i in range(1,len(search)):
        try:
            for k in range(0,len(tfperdoc[key][search[i]])):
                temp = tfperdoc[key][search[i]][k]
                if(temp-1>=0 and lokasi[temp-1]==search[i-1]):
                    dfbigram[search[i-1]+search[i]]+=1
        except Exception:
            pass

In [592]:
for key, value in (sorted(dfbigram.items(), key = itemgetter(0))):
    print key,value

iwant 2
wantyou 1


In [595]:
for key, value in (sorted(dokumen.items(), key = itemgetter(0))):
    scoreweight[key] = 0
    bigram = {}
    lokasi = {}
    kata = value.split()
    index =1 
    for i in kata:
        lokasi[index] = i
        index+=1
    for i in range(1,len(search)):
        bigram[search[i-1]+search[i]] = 0
        try:
            for k in range(0,len(tfperdoc[key][search[i]])):
                temp = tfperdoc[key][search[i]][k]
                if(temp-1>=0 and lokasi[temp-1]==search[i-1]):
                    bigram[search[i-1]+search[i]]+=1
        except Exception:
            pass
    for key1, value1 in (sorted(bigram.items(), key = itemgetter(0))):
        if(value1>0 and dfbigram[key1]>0):
            dff = math.log10(len(dokumen)/float(dfbigram[key1]))
            tff = (1+math.log10(float(value1)))
            scoreweight[key] += (tff+dff)

In [601]:
for key1, value1 in (sorted(scoreweight.items(), key = itemgetter(0),reverse=True)):
    if(value1>0):
        print "dokumen : ",key1,"dengan score : ",value1

dokumen :  tes.docx dengan score :  2.65321251378
dokumen :  memasak.txt dengan score :  1.17609125906
